In [1]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier

from bs4 import BeautifulSoup
import re
import time

from nltk.corpus import stopwords
import nltk.data

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
model = Word2Vec.load('300features_40minwords_10text')
model

In [3]:
 """
 1. 숫자로 단어를 표현
 2. Word2Vec 모델은 어휘의 각 단어에 대한 feature 벡터로 구성, 'vectors'이라는 numpy list로 저장
 3. vectors의 행 수는 모델 어휘의 단어 수
 4. vectors의 열 수는 한 feature 벡터의 크기
 """
model.wv.vectors.shape

(11986, 300)

In [4]:
# 개별 단어 벡터 접근
model.wv['flower'].shape

(300,)

In [5]:
# 단어 벡터에서 k-means를 실행하고 일부 클러스터를 찍어봄
start = time.time()

# 클러스터의 크기 "k"를 어휘 크기의 1/5나 평균 5단어로 설정
word_vectors = model.wv.vectors
num_clusters = word_vectors.shape[0] // 5

# K-means를 정의하고 학습
kmeans_clustering = KMeans(n_clusters=num_clusters)
idx = kmeans_clustering.fit_predict(word_vectors)

# 프로세싱 시간을 구함
end = time.time()
elapsed = end-start
print("Time taken for K-means clustering: ", elapsed, "seconds.")

/Users/jiwoonwon/anaconda3/lib/python3.6/site-packages/sklearn/metrics/pairwise.py:257: RuntimeWarning: invalid value encountered in sqrt
  return distances if squared else np.sqrt(distances, out=distances)


Time taken for K-means clustering:  226.3507058620453 seconds.


In [6]:
# 각 어휘 단어를 클러스터 번호에 매핑되도록 word/Index 사전을 생성
idx = list(idx)
names = model.wv.index2word
word_centroid_map = {names[i]: idx[i] for i in range(len(names))}

# 첫 번째 클러스터의 처음 10개를 출력
for cluster in range(10):
    # 클러스터 번호 출력
    print("\nCluster {}".format(cluster))
    
    # 클러스터 번호와 클러스터에 있는 단어를 찍음
    words = []
    for i in range(len(list(word_centroid_map.values()))):
        if list(word_centroid_map.values())[i] == cluster:
            words.append(list(word_centroid_map.keys())[i])
    
    print(words)


Cluster 0
['steven', 'spielberg', 'segal', 'donner', 'mcg']

Cluster 1
['forg']

Cluster 2
['shaft', 'clutch', 'skeleton', 'pinch', 'portal', 'hid', 'lemon', 'skyscrap', 'suitcas', 'dumpster', 'crate']

Cluster 3
['dick', 'lou', 'hawk', 'gregori', 'bud', 'randi', 'evan', 'dana', 'spencer', 'ethan', 'shepherd', 'duncan', 'carson', 'wheeler', 'moreland', 'morrow', 'corki', 'hackett', 'chum']

Cluster 4
['event', 'incid', 'occurr']

Cluster 5
['ach', 'sooth', 'etch']

Cluster 6
['north', 'africa', 'coast', 'pacif', 'northern', 'tokyo', 'patrol', 'atlant', 'iceland', 'northwest', 'alaska', 'carolina', 'pickup', 'gilligan', 'submerg', 'mediterranean', 'southwest']

Cluster 7
['detach', 'disconnect']

Cluster 8
['logo', 'catalog', 'billboard']

Cluster 9
['tale', 'fabl', 'propheci', 'fairytal', 'relic', 'backbon', 'parabl', 'lore', 'docudrama']


In [7]:
train = pd.read_csv('../data/imdb/labeledTrainData.tsv', 
                    header=0, delimiter="\t", quoting=3)
test = pd.read_csv('../data/imdb/testData.tsv', 
                   header=0, delimiter="\t", quoting=3)

In [8]:
from KaggleWord2VecUtility import KaggleWord2VecUtility

clean_train_reviews = []
for review in train['review']:
    clean_train_reviews.append(
        KaggleWord2VecUtility.review_to_wordlist(review, remove_stopwords=True))

In [9]:
clean_test_reviews = []
for review in test['review']:
    clean_test_reviews.append(
        KaggleWord2VecUtility.review_to_wordlist(review, remove_stopwords=True)
    )

In [10]:
train_centroids = np.zeros((train['review'].size, num_clusters), dtype='float32')
train_centroids[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [11]:
def create_bag_of_centroids(wordlist, word_centroid_map):
    """
    centroid는 두 클러스터의 중심점을 정의한 다음 중심점의 거리를 측정한 것
    """
    # 클러스터의 수는 word/centroid map 에서 가장 높은 클러스터 index와 같다
    num_centroids = max(word_centroid_map.values()) + 1
    
    bag_of_centroids = np.zeros(num_centroids, dtype='float32')
    
    # 단어가 word_centroid_map에 있다면 클러스터의 수를 1개씩 증가
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
            
    return bag_of_centroids

In [12]:
counter = 0
for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids(review, word_centroid_map)
    counter += 1
    
test_centroids = np.zeros((test['review'].size, num_clusters), dtype='float32')

counter = 0
for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids(review, word_centroid_map)
    counter += 1
    
# 랜덤 포레스트를 사용하여 학습시키고 예측
forest = RandomForestClassifier(n_estimators=100)

# train 데이터의 레이블을 통해 학습시키고 예측
print("Fitting a random forest to labeled training data")
%time forest = forest.fit(train_centroids, train['sentiment'])

Fitting a random forest to labeled training data
CPU times: user 35.7 s, sys: 212 ms, total: 35.9 s
Wall time: 36 s


In [13]:
from sklearn.model_selection import cross_val_score
%time score = np.mean(cross_val_score(forest, train_centroids, train['sentiment'],\
                                     cv=10, scoring='roc_auc'))

CPU times: user 5min 5s, sys: 3.14 s, total: 5min 8s
Wall time: 5min 9s


In [14]:
%time result = forest.predict(test_centroids)

CPU times: user 1.36 s, sys: 14.1 ms, total: 1.37 s
Wall time: 1.39 s


In [15]:
score

0.9163154880000001

In [17]:
# 결과를 csv로 저장
output = pd.DataFrame(data={'id': test['id'], 'sentiment': result})
output.to_csv("../data/imdb/submit_BagOfCentroids_{0:.5f}.csv".format(score), index=False, quoting=3)

In [18]:
output_sentiment = output['sentiment'].value_counts()
print(output_sentiment[0] - output_sentiment[1])
output_sentiment

412


0    12706
1    12294
Name: sentiment, dtype: int64